# Get data
```
mkdir -p /home/paul/cmet/data/20200909-maclaren-fs-7.1-samseg-and-aseg/
cd /home/paul/cmet/data/20200909-maclaren-fs-7.1-samseg-and-aseg/
aws s3 cp s3://cmet-scratch/maclaren-cmeds/demographics.tsv .
aws s3 cp \
  --recursive \
  --exclude "*" \
  --include "*.stats" \
  s3://cmet-scratch/20200909-maclaren-fs-7.1-samseg-and-aseg/ .
```

In [21]:
import json
import os
import fnmatch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# imports find_json_files(); load_json_file(); load_dataset(); load_fs_dataset()
from cmeds import *
# imports calc_cvs(); session_permute(); monte_carlo_perm_test
from test_retest import *

In [22]:
structs_of_interest = [
    'Left-Lateral-Ventricle',
    'Left-Hippocampus',
    'Left-Amygdala',
    'Left-Caudate',
    'Left-Putamen',
    'Right-Lateral-Ventricle',
    'Right-Hippocampus',
    'Right-Amygdala',
    'Right-Caudate',
    'Right-Putamen'
]
# Add some lateral regions together so we can directly compare to table 1 in MacLaren et al.
regions = [             
            [ ['Left-Hippocampus', 'Right-Hippocampus'],'Hippocampus' ],
            [ ['Left-Lateral-Ventricle', 'Right-Lateral-Ventricle'],'Lateral-Ventricles' ],
            [ ['Left-Amygdala', 'Right-Amygdala',],'Amygdala' ],
            [ ['Left-Putamen', 'Right-Putamen'],'Putamen' ],
            [ ['Left-Caudate', 'Right-Caudate'],'Caudate' ],
          ]
drop_subjects = []
demofile = '/home/paul/cmet/data/20200909-maclaren-fs-7.1-samseg-and-aseg/demographics.tsv'
datadir = '/home/paul/cmet/data/20200909-maclaren-fs-7.1-samseg-and-aseg//'
maclaren_fs_vol_aseg_df = load_fs_dataset(datadir, demofile, structs_of_interest, drop_subjects=[])
maclaren_fs_vol_aseg_df = add_regions(maclaren_fs_vol_aseg_df,regions)

maclaren_fs_vol_samseg_df = load_fssamseg_dataset(datadir, demofile, structs_of_interest, drop_subjects=[])
maclaren_fs_vol_samseg_df = add_regions(maclaren_fs_vol_samseg_df,regions)

Dropping the following subjects []
Dropping the following subjects []


In [26]:
# The column name that holds session info in the demographics.tsv
session_col='session'
# The column name that holds subject info in the demographics.tsv
subject_col='subject_num'
structs_of_interest = ['Hippocampus', 'Lateral-Ventricles', 'Amygdala', 'Putamen', 'Caudate']


session_list= [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
subject_list= [1,2,3]

n = 1000
perm_test_reults_aseg = monte_carlo_perm_test(maclaren_fs_vol_aseg_df, subject_list, session_list, subject_col, session_col, structs_of_interest, n_itrs=n, method='gluer')
perm_test_reults_samseg = monte_carlo_perm_test(maclaren_fs_vol_samseg_df, subject_list, session_list, subject_col, session_col, structs_of_interest, n_itrs=n, method='gluer')


Comparable to [Table 1 in MacLaren et al](https://www.nature.com/articles/sdata201437/tables/2)

In [27]:
perm_test_reults_aseg

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,8588.422500,12256.834167,3564.247500,9287.095000,7050.834167
total-cov,1.737995,2.381960,2.922279,2.458737,1.855322
session-cov,1.874242,0.927510,3.078659,2.290151,1.657972
abs-diff-cov,0.136247,1.454450,0.156380,0.168586,0.197350
p-vals,0.234000,0.000000,0.437000,0.426000,0.107000


In [28]:
perm_test_reults_samseg

,Hippocampus,Lateral-Ventricles,Amygdala,Putamen,Caudate
mean-vol,10507.386936,20527.349246,3798.720969,12428.730026,8298.407936
total-cov,2.477247,6.002172,1.860906,9.007040,3.395258
session-cov,2.422750,5.824028,1.913660,9.603341,3.445871
abs-diff-cov,0.054497,0.178144,0.052753,0.596301,0.050613
p-vals,0.715000,0.657000,0.685000,0.304000,0.810000
